# 一、百度地图API数据爬取代码 

- 本篇论文由于时间、爬取成本等原因，没有用到此代码亲自爬取数据，但仍然提供此代码。

In [1]:
#导入工具包
import os
import datetime
import time 
import requests
import math
import json
import sys
from openpyxl import Workbook
import schedule

In [2]:
#url_list记录了SEMs关注的道路的名称
url_list = ['天台路', '环城西路北段', '丰产路', '西三环辅路', '太白南路', '大兴东路','星火路', '西关正街', '西二环路', '二环南路西段', '太白北路', '长安中路', '朱雀大街南段辅路', '朱雀大街南段', '红专南路', '昌明路', '未央路', '昆明路', '丰镐西路', '丰镐东路', '劳动路', '劳动南路', '环城西路', '丰庆路', '北关正街', '自强东路', '自强西路', '太华南路', '莲湖路', '西五路', '东五路', '长乐西路', '长乐中路', '柿园路', '互助路', '韩森路', '环城南路', '咸宁西路', '咸宁中路', '环城东路', '太乙路', '小寨东路', '小寨西路', '友谊东路', '友谊西路', '兴庆路', '东二环路', '太白北路', '朱雀大街', '含光路', '雁塔北路', '雁塔南路', '长安北路', '长安中路', '长安南路', '阿房路', '团结西路', '团结中路', '团结东路', '丰禾路','振华路', '二马路', '东风路', '凤城四路', '凤城三路', '凤城二路', '凤城一路', '明光路', '文景路','玄武路', '工农路', '西北三路', '东新街', '西华门大街','含元路', '东元路', '长缨东路', '长缨西路','纺织城西街', '纺织城东街', '朱宏路', '太乙路','曲江路']
for i in url_list:
    url='https://api.map.baidu.com/traffic/v1/road?road_name=%s&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga'%i
    print(url) 

https://api.map.baidu.com/traffic/v1/road?road_name=天台路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=环城西路北段&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=丰产路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=西三环辅路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=太白南路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=大兴东路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=星火路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=西关正街&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=西二环路&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga
https://api.map.baidu.com/traffic/v1/road?road_name=二环南路西段&city=西安市&ak=nivtq4A9mRT

In [3]:
#请求url方法get方法
def request_url_get(url):
    try:
        r = requests.get(url=url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1 Win64;x64;rv:63.0) Gecko/20100101 Firefox/63.0"},timeout=30)
        if r.status_code == 200:
            return r.text
        return None
    except requests.exceptions.ConnectionError:
        print('请求url返回错误异常')
        return None

In [4]:
#解析json函数
def parse_json(content_json):
    result_json = json.loads(content_json)
    return result_json

In [5]:
#请求百度api解析json
def request_api(url):
    result = request_url_get(url)
    result_json = parse_json(result)
    return result_json

In [6]:
#定义爬取函数
def myjob():
    evaluation_status = []
    conges_sec_status=[]
    conges_sec_trend=[]
    acquire_time = []
    for i in range(12):
        for url in url_list:
            try:
                acquire_time = acquire_time + [1020+15*(i+1)]
                result = request_api('https://api.map.baidu.com/traffic/v1/road?road_name=%s&city=西安市&ak=nivtq4A9mRT1IalzXg2NFU4HOEvvBzga'%url)
                try:
                    new_evaluation_status = result['evaluation']['status']
                    evaluation_status=evaluation_status+[new_evaluation_status]
                    new_conges_sec_status = result['road_traffic'][0]['congestion_sections'][0]['status']
                    conges_sec_status = conges_sec_status + [new_conges_sec_status]
                    new_conges_sec_trend = result['road_traffic'][0]['congestion_sections'][0]['congestion_trend']
                    conges_sec_trend = conges_sec_trend + [new_conges_sec_trend]
                except:
                    evaluation_status = evaluation_status + [1]
                    conges_sec_status = conges_sec_status + [1]
                    conges_sec_trend = conges_sec_trend + ['None']
            except:
                evaluation_status = evaluation_status + ['wrong!']
                conges_sec_status = conges_sec_status + ['wrong!']
                conges_sec_trend = conges_sec_trend + ['None']
                print('wrong!')
                continue
        print(i)
        time.sleep(900)
    wb = Workbook()
    sheet1 = wb.active
    sheet1.title = 'road_condition'
    col_name = ['evaluation_status','conges_sec_status','conges_sec_trend','acquire_time']
    variable_name = [evaluation_status,conges_sec_status,conges_sec_trend,acquire_time]
    for j in range(1,len(col_name)+1):
        sheet1.cell(1,j,col_name[j-1])
        for i in range(2,len(variable_name[j-1])+2):
            sheet1.cell(i,j,variable_name[j-1][i-2])
    wb.save('road_condition.xlsx')
    print('已保存')

In [7]:
#设置定时程序，在每天的08:00开始运行程序myjob
schedule.every().day.at('08:00').do(myjob)

Every 1 day at 17:00:00 do myjob() (last run: [never], next run: 2022-10-20 17:00:00)

In [ ]:
#当目前时间与定时一致时，开始执行程序
while True:
    schedule.run_pending()
    time.sleep(1)

0
1
chucuole
请求url返回错误异常
chucuole
chucuole
2
3
4
5
6
7
8
9
10
11
已保存


# 二、深圳数据开发平台API数据获取

- 街道实时数据： 
- https://opendata.sz.gov.cn/data/dataSet/toDataDetails/29200_00403589
- 数据从2018年3月1日到11月24日，共5,546,000条，API端口共分为555页储存，其中每页10000条数据

In [2]:
#导入工具包
import os
import datetime
import time 
import requests
import math
import json
import sys
from openpyxl import Workbook
import schedule

In [3]:
#请求url方法get方法
def request_url_get(url):
    try:
        r = requests.get(url=url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1 Win64;x64;rv:63.0) Gecko/20100101 Firefox/63.0"},timeout=30)
        if r.status_code == 200:
            return r.text
        return None
    except requests.exceptions.ConnectionError:
        print('请求url返回错误异常')
        return None

In [4]:
#解析json函数
def parse_json(content_json):
    result_json = json.loads(content_json)
    return result_json

In [5]:
#请求百度api解析json
def request_api(url):
    result = request_url_get(url)
    result_json = parse_json(result)
    return result_json

In [40]:
#rows至多为10000条,共5,546,000条，共555页
url='https://opendata.sz.gov.cn/api/29200_00403589/1/service.xhtml?page=278&rows=10000&appKey=238f879cf5e0476886065bcb3b280d37'
data = request_api(url)

In [41]:
data

{'total': 5377676,
 'data': [{'GOLEN': 214997.0,
   'SPEED': 23.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 279,
   'EXPONENT': 4.0,
   'BLOCKID': 19,
   'GOTIME': 33375.0},
  {'GOLEN': 269407.0,
   'SPEED': 25.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 279,
   'EXPONENT': 3.0,
   'BLOCKID': 17,
   'GOTIME': 38359.0},
  {'GOLEN': 447877.0,
   'SPEED': 43.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 279,
   'EXPONENT': 2.0,
   'BLOCKID': 52,
   'GOTIME': 37400.0},
  {'GOLEN': 407723.0,
   'SPEED': 38.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 279,
   'EXPONENT': 2.0,
   'BLOCKID': 7,
   'GOTIME': 38799.0},
  {'GOLEN': 97177.0,
   'SPEED': 42.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 279,
   'EXPONENT': 2.0,
   'BLOCKID': 63,
   'GOTIME': 8383.0},
  {'GOLEN': 425578.0,
   'SPEED': 54.0,
   'TIME1': '2018-07-15 00:00:00',
   'PERIOD': 280,
   'EXPONENT': 2.0,
   'BLOCKID': 53,
   'GOTIME': 28219.0},
  {'GOLEN': 132451.0,
   'SPEED': 39.0,
   'TIME1': 

In [42]:
#爬取数据
TIME1 = []
PERIOD = []
EXPONENT = []
BLOCKID = []
for j in range(1,555):
    url='https://opendata.sz.gov.cn/api/29200_00403589/1/service.xhtml?page=%s&rows=10000&appKey=238f879cf5e0476886065bcb3b280d37'%j
    data = request_api(url)
    for i in range(len(data['data'])):
        newTIME1 = data['data'][i]['TIME1']
        newPERIOD = data['data'][i]['PERIOD']
        newEXPONENT = data['data'][i]['EXPONENT']
        newBLOCKID = data['data'][i][ 'BLOCKID']
        TIME1 = TIME1 + [newTIME1]
        PERIOD = PERIOD + [newPERIOD]
        EXPONENT = EXPONENT + [newEXPONENT]
        BLOCKID = BLOCKID + [newBLOCKID]
    print(j)

277
278
279
280


In [43]:
#将数据写入到Excel并保存
wb = Workbook()
sheet1 = wb.active
sheet1.title = 'road_condition'
col_name = ['TIME1','PERIOD','EXPONENT','BLOCKID']
variable_name = [TIME1,PERIOD,EXPONENT,BLOCKID]
for j in range(1,len(col_name)+1):
    sheet1.cell(1,j,col_name[j-1])
    for i in range(2,len(variable_name[j-1])+2):
        sheet1.cell(i,j,variable_name[j-1][i-2])
wb.save('深圳数据集1-555.xlsx')
print('已保存')

已保存
